# Ensemble Learning

## Initial Imports

In [1]:
# ignore warnings
import warnings
warnings.filterwarnings('ignore')

# tools
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# data processing/sampling
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

# model/metrics
from imblearn.ensemble import BalancedRandomForestClassifier
from imblearn.ensemble import EasyEnsembleClassifier
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced


## Read the CSV and Perform Basic Data Cleaning

In [2]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Preview the data
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


# Perform encoding on non neumerical values

In [3]:
# init label encoder
le = LabelEncoder()

# create new df copy for use in X dataset
encoded_df = df.copy()

# find columns where datatype is not an int or float,
# encode values using label encoder and apply encodings to encoded_df
for column in df.select_dtypes(exclude=['float64','int64']):
        le.fit(df[column])
        encoded_df[column] = le.transform(df[column])

encoded_df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,3,66000.0,1,2,1,0,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,0,0
1,25000.0,0.2000,929.09,1,105000.0,2,2,1,0,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,0,0
2,20000.0,0.2000,529.88,1,56000.0,2,2,1,0,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,0,0
3,10000.0,0.1640,353.55,3,92000.0,2,2,1,0,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,0,0
4,22000.0,0.1474,520.39,1,52000.0,0,2,1,0,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,0,0


In [4]:
# Create our features, drop target
X = encoded_df.drop(columns=['loan_status'])

# Create and encode our target (-1 to set 0=low_risk and 1=high_risk)
le.fit(df['loan_status'])
y = 1-le.transform(df['loan_status'])

In [5]:
# Check the balance of our target value
Counter(y)

Counter({0: 68470, 1: 347})

## Split the Data into Training and Testing

In [6]:
# Split the X and y into X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [7]:
# Create the StandardScaler instance
scaler = StandardScaler()

In [8]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
scaler.fit(X_train)

StandardScaler()

In [9]:
# Scale the training and testing data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [10]:
# Resample the training data with the BalancedRandomForestClassifier
rf_balanced = BalancedRandomForestClassifier(random_state=1, n_estimators=100)
rf_balanced.fit(X_train_scaled, y_train)

BalancedRandomForestClassifier(random_state=1)

In [11]:
# Calculated the balanced accuracy score
y_pred = rf_balanced.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred)

0.7519345830747715

In [12]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)
# Display the confusion matrix [[tn,fp]
#                               [fn,tp]]
cm = pd.DataFrame(confusion_matrix(y_test, y_pred))
cm = cm.rename(columns={0:'low_risk (pred)', 1:'high_risk (pred)'}, index={0:'low_risk (true)', 1:'high_risk (true)'})
cm

,low_risk (pred),high_risk (pred)
low_risk (true),14884,2220
high_risk (true),37,64


In [13]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test,y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.87      0.63      0.93      0.74      0.56     17104
          1       0.03      0.63      0.87      0.05      0.74      0.54       101

avg / total       0.99      0.87      0.64      0.92      0.74      0.56     17205



In [14]:
# List the features sorted in descending order by feature importance
feature_imp = pd.DataFrame(sorted(zip(rf_balanced.feature_importances_, X.columns), reverse=True))
feature_imp = feature_imp.rename(columns={0:'importance', 1:'feature'}).set_index('feature')

# show top ten features
feature_imp.iloc[:10]

,importance
feature,
total_rec_prncp,0.083176
last_pymnt_amnt,0.069083
total_pymnt,0.056675
total_pymnt_inv,0.055934
total_rec_int,0.045119
int_rate,0.033150
issue_d,0.023794
installment,0.017899
dti,0.016902


### Easy Ensemble Classifier

In [15]:
# Train the Classifier
easy_ensemble = EasyEnsembleClassifier(random_state=1, n_estimators=100)
easy_ensemble.fit(X_train_scaled, y_train)

EasyEnsembleClassifier(n_estimators=100, random_state=1)

In [16]:
# Calculated the balanced accuracy score
y_pred = easy_ensemble.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred)

0.9316600714093861

In [19]:
# Display the confusion matrix [[tn,fp]
#                               [fn,tp]]
cm = pd.DataFrame(confusion_matrix(y_test, y_pred))
cm = cm.rename(columns={0:'low_risk (pred)', 1:'high_risk (pred)'}, index={0:'low_risk (true)', 1:'high_risk (true)'})
cm

,low_risk (pred),high_risk (pred)
low_risk (true),16121,983
high_risk (true),8,93


In [18]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test,y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.94      0.92      0.97      0.93      0.87     17104
          1       0.09      0.92      0.94      0.16      0.93      0.87       101

avg / total       0.99      0.94      0.92      0.97      0.93      0.87     17205



### Final Questions

1. Which model had the best balanced accuracy score?

The easy ensemble classifier performed significantly better than the balanced random forest, with a balanced accuracy score of 0.93

2. Which model had the best recall score?

Again, the easy ensemble classifier, performed better at minimising incorrect predictions of high risk loans with a recall of 0.94

3. Which model had the best geometric mean score?

A geometric mean of 0.93 for the easy ensemble classifier places it firmly above the balanced random forest classifier.

4. What are the top three features?

The three most important features according to the balanced random forest classifier were:
- total_rec_prncp
- last_pymnt_amnt
- total_pymnt